In [1]:
from pynq import Overlay, Xlnk
import pynq.lib.dma
import numpy as np
overlay = Overlay('/home/xilinx/jupyter_notebooks/overlay/mlp.bit')
Xlnk.set_allocator_library('/home/xilinx/jupyter_notebooks/overlay/mlp.so')

In [2]:
image_size = 28*28
l1_w_size = 784*100
l1_b_size = 100
l2_w_size = 100*10
l2_b_size = 10
output_size = 10

In [3]:
image_np = np.load('test_set/one.npy')
image_np = (image_np * 16).astype(int)
image_np = image_np.flatten()

l1_w_np = np.load('weights/l1_w.npy')
l1_w_np = (l1_w_np * 16).astype(int)
l1_w_np = l1_w_np.flatten()

l1_b_np = np.load('weights/l1_b.npy')
l1_b_np = (l1_b_np * 16).astype(int)

l2_w_np = np.load('weights/l2_w.npy')
l2_w_np = (l2_w_np * 16).astype(int)
l2_w_np = l2_w_np.flatten()

l2_b_np = np.load('weights/l2_b.npy')
l2_b_np = (l2_b_np * 16).astype(int)

In [4]:
import cffi
ffi = cffi.FFI()
ffi.cdef("void p_0_MLP_1_noasync(unsigned char *image, signed char *l1_w, signed char *l1_b, signed char *l2_w, signed char *l2_b, unsigned char *out);")
dlib = ffi.dlopen('/home/xilinx/jupyter_notebooks/overlay/mlp.so')

In [5]:
xlnk = Xlnk()
image = xlnk.cma_array(shape=(image_size,), dtype=np.uint8) #(int *) sds_alloc(image_size_bytes);
l1_w = xlnk.cma_array(shape=(l1_w_size,), dtype=np.int8) #(int *) sds_alloc(weight_size_bytes);
l1_b = xlnk.cma_array(shape=(l1_b_size,), dtype=np.int8) #(int *) sds_alloc(weight_size_bytes);
l2_w = xlnk.cma_array(shape=(l2_w_size,), dtype=np.int8) #(int *) sds_alloc(weight_size_bytes);
l2_b = xlnk.cma_array(shape=(l2_b_size,), dtype=np.int8) #(int *) sds_alloc(weight_size_bytes);
output = xlnk.cma_array(shape=(output_size,), dtype=np.uint8) #(int *) sds_alloc(output_size_bytes);


In [6]:
for i in range(image_size):
    image[i] = image_np[i]

for i in range(l1_w_size):
    l1_w[i] = l1_w_np[i]

for i in range(l1_b_size):
    l1_b[i] = l1_b_np[i]
    
for i in range(l2_w_size):
    l2_w[i] = l2_w_np[i]

for i in range(l2_b_size):
    l2_b[i] = l2_b_np[i]

for i in range(output_size): #(int i = 0; i < o_chan*OSize*OSize; i++)
    output[i] = 0
    

In [7]:
dlib.p_0_MLP_1_noasync(image.pointer, l1_w.pointer, l1_b.pointer, l2_w.pointer, l2_b.pointer, output.pointer)

In [8]:
output = output / 16
output = output * 100


In [15]:
print("Results")
for i in range(output_size):
    print(str(i) + " " + np.array2string(output[i]) + '%')

Results
0 0.0%
1 100.0%
2 0.0%
3 0.0%
4 0.0%
5 0.0%
6 0.0%
7 0.0%
8 0.0%
9 0.0%


In [16]:
xlnk.xlnk_reset()